In [33]:
from google.colab import files
import numpy as np
import pandas as pd
import io
import re
from string import punctuation
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
import warnings
warnings.filterwarnings("ignore")

In [5]:
uploaded = files.upload()


Saving amazon.csv to amazon.csv


In [20]:
df = pd.read_csv(io.BytesIO(uploaded['amazon.csv']))
df

,product_id,product_name,category,discounted_price,actual_price,discount_percentage,rating,rating_count,about_product,user_id,user_name,review_id,review_title,review_content,img_link,product_link
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,₹399,"₹1,099",64%,4.2,"24,269",High Compatibility : Compatible With iPhone 12...,"AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jasp...","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-WN3LG1-Sy...
1,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,Computers&Accessories|Accessories&Peripherals|...,₹199,₹349,43%,4.0,"43,994","Compatible with all Type C enabled devices, be...","AECPFYFQVRUWC3KGNLJIOREFP5LQ,AGYYVPDD7YG7FYNBX...","ArdKn,Nirbhay kumar,Sagar Viswanathan,Asp,Plac...","RGIQEG07R9HS2,R1SMWZQ86XIN8U,R2J3Y1WL29GWDE,RY...","A Good Braided Cable for Your Type C Device,Go...",I ordered this cable to connect my phone to An...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Ambrane-Unbreakable-Char...
2,B096MSW6CT,Sounce Fast Phone Charging Cable & Data Sync U...,Computers&Accessories|Accessories&Peripherals|...,₹199,"₹1,899",90%,3.9,"7,928",【 Fast Charger& Data Sync】-With built-in safet...,"AGU3BBQ2V2DDAMOAKGFAWDDQ6QHA,AESFLDV2PT363T2AQ...","Kunal,Himanshu,viswanath,sai niharka,saqib mal...","R3J3EQQ9TZI5ZJ,R3E7WBGK7ID0KV,RWU79XKQ6I1QF,R2...","Good speed for earlier versions,Good Product,W...","Not quite durable and sturdy,https://m.media-a...",https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Sounce-iPhone-Charging-C...
3,B08HDJ86NZ,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,Computers&Accessories|Accessories&Peripherals|...,₹329,₹699,53%,4.2,"94,363",The boAt Deuce USB 300 2 in 1 cable is compati...,"AEWAZDZZJLQUYVOVGBEUKSLXHQ5A,AG5HTSFRRE6NL3M5S...","Omkar dhale,JD,HEMALATHA,Ajwadh a.,amar singh ...","R3EEUZKKK9J36I,R3HJVYCLYOY554,REDECAZ7AMPQC,R1...","Good product,Good one,Nice,Really nice product...","Good product,long wire,Charges good,Nice,I bou...",https://m.media-amazon.com/images/I/41V5FtEWPk...,https://www.amazon.in/Deuce-300-Resistant-Tang...
4,B08CF3B7N1,Portronics Konnect L 1.2M Fast Charging 3A 8 P...,Computers&Accessories|Accessories&Peripherals|...,₹154,₹399,61%,4.2,"16,905",[CHARGE & SYNC FUNCTION]- This cable comes wit...,"AE3Q6KSUK5P75D5HFYHCRAOLODSA,AFUGIFH5ZAFXRDSZH...","rahuls6099,Swasat Borah,Ajay Wadke,Pranali,RVK...","R1BP4L2HH9TFUP,R16PVJEXKV6QZS,R2UPDB81N66T4P,R...","As good as original,Decent,Good one for second...","Bought this instead of original apple, does th...",https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Portronics-Konnect-POR-1...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1460,B08L7J3T31,Noir Aqua - 5pcs PP Spun Filter + 1 Spanner | ...,Home&Kitchen|Kitchen&HomeAppliances|WaterPurif...,₹379,₹919,59%,4,"1,090",SUPREME QUALITY 90 GRAM 3 LAYER THIK PP SPUN F...,"AHITFY6AHALOFOHOZEOC6XBP4FEA,AFRABBODZJZQB6Z4U...","Prabha ds,Raghuram bk,Real Deal,Amazon Custome...","R3G3XFHPBFF0E8,R3C0BZCD32EIGW,R2EBVBCN9QPD9R,R...","Received the product without spanner,Excellent...","I received product without spanner,Excellent p...",https://m.media-amazon.com/images/I/41fDdRtjfx...,https://www.amazon.in/Noir-Aqua-Spanner-Purifi...
1461,B01M6453MB,Prestige Delight PRWO Electric Rice Cooker (1 ...,Home&Kitchen|Kitchen&HomeAppliances|SmallKitch...,"₹2,280","₹3,045",25%,4.1,"4,118","230 Volts, 400 watts, 1 Year","AFG5FM3NEMOL6BNFRV2NK5FNJCHQ,AGEINTRN6Z563RMLH...","Manu Bhai,Naveenpittu,Evatira Sangma,JAGANNADH...","R3DDL2UPKQ2CK9,R2SYYU1OATVIU5,R1VM993161IYRW,R...","ok,everything was good couldn't return bcoz I ...","ok,got everything as mentioned but the me

In [18]:
#dfReview for review title and review content
dfReview = df[['review_title','review_content','about_product']]
dfReview

,review_title,review_content,about_product
0,"Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...,High Compatibility : Compatible With iPhone 12...
1,"A Good Braided Cable for Your Type C Device,Go...",I ordered this cable to connect my phone to An...,"Compatible with all Type C enabled devices, be..."
2,"Good speed for earlier versions,Good Product,W...","Not quite durable and sturdy,https://m.media-a...",【 Fast Charger& Data Sync】-With built-in safet...
3,"Good product,Good one,Nice,Really nice product...","Good product,long wire,Charges good,Nice,I bou...",The boAt Deuce USB 300 2 in 1 cable is compati...
4,"As good as original,Decent,Good one for second...","Bought this instead of original apple, does th...",[CHARGE & SYNC FUNCTION]- This cable comes wit...
...,...,...,...
1460,"Received the product without spanner,Excellent...","I received product without spanner,Excellent p...",SUPREME QUALITY 90 GRAM 3 LAYER THIK PP SPUN F...
1461,"ok,everything was good couldn't return bcoz I ...","ok,got everything as mentioned but the measuri...","230 Volts, 400 watts, 1 Year"
1462,"very good,Work but front melt after 2 month,Go...","plastic but cool body ,u have to find sturdy s...",International design and styling|Two heat sett...
1463,"Fan Speed is slow,Good quality,Good product,go...",I have installed this in my kitchen working fi...,Fan sweep area: 230 MM ; Noise level: (40 - 45...


In [34]:
#replace capital letters and remove punctuation
for column in range(3):
  dfReview.iloc[:,column] = dfReview.iloc[:,column].apply(lambda x: x.lower())
  dfReview.iloc[:,column] = dfReview.iloc[:,column].apply((lambda x: re.sub(r"http\S+"," ",x)))
  dfReview.iloc[:,column] = dfReview.iloc[:,column].apply((lambda x: re.sub('[^a-zA-z0-9\s]'," ",x)))
dfReview

,review_title,review_content,about_product
0,satisfied charging is really fast value for mo...,looks durable charging is fine toono complains...,high compatibility compatible with iphone 12...
1,a good braided cable for your type c device go...,i ordered this cable to connect my phone to an...,compatible with all type c enabled devices be...
2,good speed for earlier versions good product w...,not quite durable and sturdy m media amazo...,fast charger data sync with built in safet...
3,good product good one nice really nice product...,good product long wire charges good nice i bou...,the boat deuce usb 300 2 in 1 cable is compati...
4,as good as original decent good one for second...,bought this instead of original apple does th...,[charge sync function] this cable comes wit...
...,...,...,...
1460,received the product without spanner excellent...,i received product without spanner excellent p...,supreme quality 90 gram 3 layer thik pp spun f...
1461,ok everything was good couldn t return bcoz i ...,ok got everything as mentioned but the measuri...,230 volts 400 watts 1 year
1462,very good work but front melt after 2 month go...,plastic but cool body u have to find sturdy s...,international design and styling two heat sett...
1463,fan speed is slow good quality good product go...,i have installed this in my kitchen working fi...,fan sweep area 230 mm noise level 40 45...


In [32]:
#tokenize, build vocab, and add paddings on review_title
tokenizer = Tokenizer(split=' ')
tokenizer.fit_on_texts(dfReview['review_title'].values)
review_title = tokenizer.texts_to_sequences(dfReview['review_title'].values)
review_title = pad_sequences(review_title)

#tokenize, build vocab, and add paddings on review_content
tokenizer = Tokenizer(num_words=2000, split=' ')
tokenizer.fit_on_texts(dfReview['review_content'].values)
review_content = tokenizer.texts_to_sequences(dfReview['review_content'].values)
review_content = pad_sequences(review_content)

#tokenize, build vocab, and add paddings on 
tokenizer = Tokenizer(num_words=2000, split=' ')
tokenizer.fit_on_texts(dfReview['about_product'].values)
about_product = tokenizer.texts_to_sequences(dfReview['about_product'].values)
about_product = pad_sequences(about_product)

print(review_title)
print(review_content)
print(about_product)

[[  0   0   0 ... 136 252   1]
 [  0   0   0 ...  25  51 354]
 [  0   0   0 ...  75   6   2]
 ...
 [  0   0   0 ...   2   6   2]
 [  0   0   0 ...   6   2  64]
 [  0   0   0 ...  16   2   1]]
[[   0    0    0 ...  101    7   75]
 [   0    0    0 ...   12   18  508]
 [   0    0    0 ...   93   53   13]
 ...
 [   0    0    0 ...   13   53   13]
 [   0    0    0 ...   97  442  311]
 [   0    0    0 ...   56 1038   28]]
[[   0    0    0 ...    3  251  454]
 [   0    0    0 ...  143    6  108]
 [   0    0    0 ...  152   15   59]
 ...
 [   0    0    0 ... 1079  549    9]
 [   0    0    0 ... 1217 1635 1274]
 [   0    0    0 ...    1   70  464]]
